In [1]:
import os
import sys

cwd = os.getcwd()
sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from data.data_preprocessing import load_and_preprocess
from src.neural_network import NeuralNetwork

In [3]:
train, val, test = load_and_preprocess(train_sample_size=1500000, val_sample_size=300000)

In [4]:
train.head()

,label,title,text
0,1,Einstein: His Life Revealed,walter isaacson has clearly covered the bases ...
1,0,Totally unreliable measuring directions,the examples of things that can be built are v...
2,1,It's Not Just About the Book,i just finished reading the book as the tour d...
3,0,Playing in the dark,be careful buying this light the invoice and o...
4,0,"Rose-flavored Water, NOT Rosewater",if all you're after is rose flavor for cooking...


In [ ]:
print(f"Training size: {len(train)}")
print(f"Validation size: {len(val)}")
print(f"Test size: {len(test)}")

Training size: 1500000
Validation size: 300000
Test size: 400000


: 

In [ ]:
from src.functions import vectorize

X_train, vectorizer = vectorize(train["text"])
y_train = np.array(train["label"])
X_test = vectorizer.transform(test["text"])
y_test = np.array(test["label"])

In [ ]:
print(vectorizer.vocabulary_)  # shows the learned vocabulary
print("joke" in vectorizer.vocabulary_)  # should return True if "joke" exists

{'clearly': np.int64(837), 'covered': np.int64(1050), 'researching': np.int64(3700), 'fascinating': np.int64(1668), 'brilliant': np.int64(588), 'human': np.int64(2188), 'beings': np.int64(447), 'born': np.int64(547), 'got': np.int64(1946), 'lost': np.int64(2669), 'gritty': np.int64(1990), 'details': np.int64(1228), 'times': np.int64(4532), 's': np.int64(3807), 'genius': np.int64(1896), 'recommend': np.int64(3595), 'book': np.int64(537), 'interested': np.int64(2326), 'thoughts': np.int64(4504), 'life': np.int64(2584), 'relevant': np.int64(3642), 'minds': np.int64(2851), 'history': np.int64(2133), 'date': np.int64(1135), 'feel': np.int64(1693), 'gotten': np.int64(1948), 'point': np.int64(3307), 'words': np.int64(4930), 'little': np.int64(2630), 'passion': np.int64(3177), 'subject': np.int64(4310), 'examples': np.int64(1566), 'things': np.int64(4494), 'built': np.int64(621), 'interesting': np.int64(2327), 'followed': np.int64(1785), 'directions': np.int64(1272), 'chair': np.int64(738), 'w

In [ ]:
print("y_train unique:", np.unique(y_train))
print("y_train shape:", y_train.shape)
print("x_train_shape:", X_train.shape)

y_train unique: [0 1]
y_train shape: (100000,)
x_train_shape: (100000, 5000)


In [ ]:
print("y_test unique:", np.unique(y_test))
print("y_test shape:", y_test.shape)
print("x_test_shape:", X_test.shape)

y_test unique: [0 1]
y_test shape: (180000,)
x_test_shape: (180000, 5000)


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Define training function
def train_with_optimizer(optimizer_name, X_train, y_train, layer_dims, dropout_rate=0.2, l2_lambda=0.01):
    # Learning rate schedules
    if optimizer_name == "adam":
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-4, decay_steps=1000, decay_rate=0.9, staircase=True
        )
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    elif optimizer_name == "rmsprop":
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-5, decay_steps=1000, decay_rate=0.9, staircase=True
        )
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

    elif optimizer_name == "sgd":
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-4, decay_steps=500, decay_rate=0.96, staircase=True
        )
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

    elif optimizer_name == "momentum":
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-4, decay_steps=500, decay_rate=0.96, staircase=True
        )
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # Initialize model
    model = NeuralNetwork(layer_dims=layer_dims, dropout_rate=dropout_rate, l2_lambda=l2_lambda)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

    # Train
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=128,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )

    return model, history

In [ ]:
results = {}
optimizers = ["adam", "rmsprop", "sgd", "momentum"]
models = {}
layer_dims = [X_train.shape[1], 512, 256, 128, 64, 1]  # Adjust based on your input

for opt in optimizers:
    print(f"\nTraining with optimizer: {opt.upper()}")
    model, history = train_with_optimizer(
        opt, X_train, y_train,
        layer_dims=layer_dims,
        dropout_rate=0.2,
        l2_lambda=0.01
    )
    results[opt] = history
    models[opt] = model

In [ ]:
plt.figure(figsize=(10,6))
for opt in optimizers:
    plt.plot(results[opt].history['val_accuracy'], label=f'{opt} (val)')
plt.title("Validation Accuracy by Optimizer")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

for opt in optimizers:
    print(f"\nEvaluating optimizer: {opt}")
    model = models[opt]
    
    y_pred = model.predict(X_test)
    y_pred_labels = (y_pred > 0.5).astype(int)
    
    acc = accuracy_score(y_test, y_pred_labels)
    print(f"Test accuracy for {opt}: {acc:.4f}")
    